In [1]:
import pandas as pd
import re
from tqdm import tqdm

import matplotlib.pyplot as plt
import numpy as np
import locationtagger

In [2]:
df = pd.read_csv("google_review_29_08_v3.csv")
df.head()

,SVAAS_ID,user_id,firstName,middleName,lastName,email,phone,description,pan,mci,...,numberFollowUp,followUpDuration,insuranceProviderId,isActive,ABDM_Healthcare_Professional_ID,SVAAS_DOCTOR_ID,svaasClinicId,tbl_user_phone_number,name,location
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df = df[['firstName', 'lastName', 'name', 'baseCity']]
df.head()

,firstName,lastName,name,baseCity
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN


In [4]:
df_not_missing = df.dropna()

In [5]:
df_not_missing.head()

,firstName,lastName,name,baseCity
17,Divya,Kavuri,Apollo Clinic,Hyderabad
18,Shivam,P,Shivam hospitals,Hyderabad
19,Ranjan,Dey,Ranjan skin clinics,Hyderabad
20,Soumya,Sarkar,Soumya Multi speciality clinic,Hyderabad
21,Rishanku,Goyal,RKG Eye Center,Hyderabad


In [6]:
df_not_missing.isna().sum()

firstName    0
lastName     0
name         0
baseCity     0
dtype: int64

In [7]:
len(df_not_missing)/len(df)

0.9879032258064516

In [8]:
df_not_missing['query'] = df_not_missing['firstName'].apply(lambda x:x.lower()) +"+" + df_not_missing['lastName'].apply(lambda x:x.lower()) \
                          +"+"+ df_not_missing['name'].apply(lambda x:x.lower()) + "+" + df_not_missing["baseCity"].apply(lambda x:x.lower())

/var/folders/z1/0tn2v83n6pzgl2m8lzz1phbc0000gn/T/ipykernel_78132/4047833984.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_not_missing['query'] = df_not_missing['firstName'].apply(lambda x:x.lower()) +"+" + df_not_missing['lastName'].apply(lambda x:x.lower()) \


In [9]:
df_not_missing['query'].values.shape

(1715,)

## For missing data

In [10]:
import preprocess_kgptalkie as ps
import re

def get_clean(x):
    x = str(x).lower().replace('\\', '').replace('_', ' ')
    x = ps.cont_exp(x)
    x = ps.remove_emails(x)
    x = ps.remove_urls(x)
    x = ps.remove_html_tags(x)
    x = ps.remove_rt(x)
    x = ps.remove_accented_chars(x)
    x = ps.remove_special_chars(x)
    x = re.sub("(.)\\1{2,}", "\\1", x)
    return x




In [11]:
df_not_missing['query'] = df_not_missing['query'].apply(lambda x:ps.remove_accented_chars(x))
df_not_missing['query'].head()

/var/folders/z1/0tn2v83n6pzgl2m8lzz1phbc0000gn/T/ipykernel_78132/153722280.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_not_missing['query'] = df_not_missing['query'].apply(lambda x:ps.remove_accented_chars(x))


17                 divya+kavuri+apollo clinic+hyderabad
18                  shivam+p+shivam hospitals+hyderabad
19             ranjan+dey+ranjan skin clinics+hyderabad
20    soumya+sarkar+soumya multi speciality clinic+h...
21              rishanku+goyal+rkg eye center+hyderabad
Name: query, dtype: object

In [12]:
from collections import defaultdict

final_df = defaultdict(list)

In [13]:
df_not_missing = df_not_missing[['firstName', 'lastName', 'name', 'baseCity','query']]

In [14]:
def isDigit(char):
    pattern = "^[-+]?[0-9]*\.?[0-9]+(e[-+]?[0-9]+)?$"
    reg = re.compile(pattern)
    if reg.match(char):
        return True
    return False

In [15]:
# from bs4 import BeautifulSoup
# import urllib.request
# import numpy as np


# query="https://google.com/search?q=shivam+p+shivam+hospitals+hyderabad"

# request = urllib.request.Request(query)
# request.add_header('User-Agent', 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36')
# raw_response = urllib.request.urlopen(request).read()

# html = raw_response.decode("utf-8")

# soup = BeautifulSoup(html, 'html.parser')
# h3s = soup.find_all('h3')

# for h in h3s:
#     print(h.get_text())



In [16]:
"practo" in "practo.com"

True

In [17]:
from googlesearch import search
query = "see"

links = []
for j in search(query, tld="co.in", num=10, stop=10, pause=2): 
    print(j) 


https://en.wikipedia.org/wiki/See_(TV_series)
https://en.wikipedia.org/wiki/Hera_Hilmar
https://en.wikipedia.org/wiki/Nesta_Cooper
https://en.wikipedia.org/wiki/Joshua_Henry
https://en.wikipedia.org/wiki/Yadira_Guevara-Prip
https://www.youtube.com/watch?v=7Rg0y7NT1gU
https://tv.apple.com/us/show/see/umc.cmc.3s4mgg2y7h95fks9gnc4pw13m
https://www.imdb.com/title/tt7949218/
https://www.rottentomatoes.com/tv/see/s01
https://www.rottentomatoes.com/tv/see


In [23]:
from bs4 import BeautifulSoup
import urllib.request
import numpy as np
from googlesearch import search   

url = "https://www.practo.com/bangalore/doctor/dr-anjanappa-t-m-general-surgeon"
request = urllib.request.Request(url)

request.add_header('User-Agent', 'your bot 0.1')
raw_response = urllib.request.urlopen(request).read()

html = raw_response.decode("utf-8")

soup = BeautifulSoup(html, 'html.parser')

heading = soup.find("h1")
speciality = soup.find("h2", class_="u-spacer--right-thin")
rating = soup.find("div", class_="u-pos-rel")
votes = soup.find("span", attrs={"data-qa-id":"clinic-votes"})
address = soup.find("p", class_="c-profile--clinic__address")

print(heading.get_text())
# print(speciality.get_text())
print(rating.get_text())
# print(votes.get_text().strip(")").strip("("))
print(address.get_text())


Dr. Anjanappa T M
4.0
1179, A Block, 17th Cross, 6th Main, 2nd Stage, Landmark: Near to Dr. Rajkumar Road, Bangalore


In [24]:
from bs4 import BeautifulSoup
import urllib.request
import numpy as np
from googlesearch import search   


def extractReviews(df_not_missing):
    queries = df_not_missing['query'].values

    for i in tqdm(range(len(queries))):
        query = queries[i]
        query = "+".join(query.split(" "))
        url = 'https://google.com/search?q='+query

        
        for link in search(url, stop=10, pause=2, user_agent='bot 1.0'):
            if "practo" in link:
                print(link)
                request = urllib.request.Request(link)
                request.add_header('User-Agent', 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36')
                raw_response = urllib.request.urlopen(request).read()

                # Read the repsonse as a utf-8 string
                html = raw_response.decode("utf-8")

                soup = BeautifulSoup(html, 'html.parser')

                heading = soup.find("h1")
                speciality = soup.find("h2", class_="u-spacer--right-thin")
                rating = soup.find("div", class_="u-pos-rel")
                votes = soup.find("span", attrs={"data-qa-id":"clinic-votes"})
                address = soup.find("p", class_="c-profile--clinic__address")

                print(heading.get_text())
                print(speciality.get_text())
                print(rating.get_text())
                print(votes.get_text().strip(")").strip("("))
                print(address.get_text())

                heading = heading.get_text()
#                 speciality = speciality.get_text()
                rating = rating.get_text()
#                 votes = votes.get_text()
                address = address.get_text()


                final_df['name'].append(heading)
#                 final_df['speciality'].append(speciality)
                final_df['rating'].append(rating)
#                 final_df['votes'].append(votes)
                final_df['address'].append(address)
                final_df['link'].append(link)

                       
    return final_df
    #             print(div.get_text())


In [ ]:
final_df = extractReviews(df_not_missing)

  0%|                                                  | 0/1715 [00:00<?, ?it/s]

In [ ]:
pd.DataFrame(final_df).to_csv("output_v1_practo.csv", index=False)

In [ ]:
final_df

In [ ]:
df_final = pd.DataFrame(final_df)

practo: name, address, working days, phone, speciality

In [30]:
df_final

,firstName,lastName,clinicName,baseCity,practColumn,extractedCity,matched,rating
0,Ranjan,Dey,Ranjan skin clinics,Hyderabad,"Rating: 4.7 · ‎2,006 reviews",Aligarh Uttar Pradesh,False,3.9
1,Rishanku,Goyal,RKG Eye Center,Hyderabad,Rating: 5 · ‎11 votes,New Delhi Delhi,False,4.2
2,V V,Satya Sekhar,Mamata Hospital,Visakhapatnam,Rating: 3 · ‎1 vote · ‎Price range: 200 INR,Visakhapatnam Andhra Pradesh,False,3.1
3,V,Ravi Chandra,SG Ortho Care,Visakhapatnam,Rating: 5 · ‎60 votes,Visakhapatnam Andhra Pradesh,False,5.0
4,V,Chandra Sekharam,Chandra Sekharam clinic,Visakhapatnam,Rating: 4.7 · ‎10 votes,Visakhapatnam Andhra Pradesh,False,3.7
5,J K,Vijay Kumar,Urocare,Visakhapatnam,Rating: 4.6 · ‎7 votes,Visakhapatnam Andhra Pradesh,False,4.4
6,Chodisetti,Subbarao,Samudra Hospital,Visakhapatnam,Rating: 5 · ‎79 reviews · ‎Price range: INR 500,Visakhapatnam Andhra Pradesh,False,3.0
7,B S,Kalyani,Dr. B. S Kalyani Bharathi Hospital,Visakhapatnam,Rating: 5 · ‎2 votes · ‎Price range: 500 INR,Visakhapatnam Andhra Pradesh,False,3.2
8,Mohammed,Akbar,Gastro & Liver Clinic,Visakhapatnam,Rating: 4.4 · ‎11 votes · ‎Price range: 200 INR,Visakhapatnam Andhra Pradesh,False,4.1
9,Komanapalli,Sowjanya,Padmaja Hospital,Visakhapatnam,Rating: 4.2 · ‎7 votes · ‎Price range: 200 INR,Visakhapatnam Andhra Pradesh,False,3.0


In [ ]:
df.columns

In [ ]:
pd.merge(df, df_final, how='outer').to_csv("full_output_30_aug_2022.csv", index=False)

In [28]:
pd.read_csv("full_output_30_aug_2022.csv")

,firstName,lastName,name,baseCity,clinicName,GoogleNRs,rating
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1747,Kushang,Mod,Shivam Children Hospital And Neonatal Care Centre,Ahmedabad,Shivam Children Hospital And Neonatal Care Centre,18.0,2.9
1748,Dhaval,Akhani,Childhood Hospital,Ahmedabad,Childhood Hospital,436.0,4.9
1749,Keval Patel,Patel,Jk Dental Hospital Orthodontic & Implant Center,Ahmedabad,Jk Dental Hospital Orthodontic & Implant Center,12.0,5.0
1750,Jinal Patel,Patel,Jk Dental Hospital Orthodontic & Implant Center,Ahmedabad,Jk Dental Hospital Orthodontic & Implant Center,3.0,4.0


In [ ]:
df

### Processing for NA

In [29]:
df = pd.read_csv("full_output_30_aug_2022.csv")
df.head()

,firstName,lastName,name,baseCity,clinicName,GoogleNRs,rating
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
df_na = df[df['rating'].isna() == True]

In [36]:
df_na = df_na[df_na['firstName'].isna()!=True].drop('clinicName', axis=1)

In [39]:
df_na.columns = ['firstName', 'lastName', 'name', 'baseCity', 'GoogleNRs', 'googleRating']

In [40]:
df_na

,firstName,lastName,name,baseCity,GoogleNRs,googleRating
18,Divya,Kavuri,Apollo Clinic,Hyderabad,NaN,NaN
19,Shivam,P,Shivam hospitals,Hyderabad,NaN,NaN
23,Mamatha,R,Mamatha Clinics,Hyderabad,NaN,NaN
25,B,Ramesh,Aditya Medical Complex,Visakhapatnam,NaN,NaN
30,Sreeniavsa Rao,Reddy,Sri Srinivasa Gastro & Liver Centre,Visakhapatnam,NaN,NaN
...,...,...,...,...,...,...
1727,Dhirena,Sanandiya,Vaibhav Clinic,Ahmedabad,NaN,NaN
1728,Rakesh,Sanol,Sadbhavna Hospital,Ahmedabad,NaN,NaN
1735,Lal,Daga,Dr LalS Heart Centre,Ahmedabad,NaN,NaN
1744,Narayan,Varma,Pakhi Hospital,Ahmedabad,NaN,NaN
